Primative: $\rho_0,\ u,\ \mathcal{B}^i,\ \tilde{u}^i$ </br>
Conserved: $Q_\mu,\ D,\ \mathcal{B}^i$ </br>
EOS: $p = (\Gamma - 1) u$


In [277]:
import numpy as np
import matplotlib.pyplot as plt

In [545]:
def newton(func, x0, tol=1e-9, max_iter=1000, dx=1e-12):
        
    dfunc = lambda x: (func(x + dx) - func(x - dx)) / (2 * dx)
    
    x = x0
    
    for i in range(max_iter):
        
        x_old = x
        x = x - func(x) / dfunc(x)
        
        if x == 0: 
            errx = np.abs(x - x_old)
        else: 
            errx = np.abs(x - x_old) / x
            
        if errx < tol: 
            print("Newton's method took %d iterations to converge." % i)
            break

    else:
    
        print("Newton's method did not converge!")
            
    return x


In [592]:
# define metric

gcov = np.diag([-1, 1, 1, 1]) # covariant metric tensor
gcon = np.linalg.inv(gcov) # contravariant metric tensor
gdet = np.linalg.det(gcov) # metric tensor determinate


In [614]:
# define tensor operations

COV, MIX, CON = 0, 1, 2

def raize(arr): 
    ''' Raise a tensor index ''' 
    return np.tensordot(gcon, arr, axes=(1, 0))

def lower(arr): 
    ''' Lower a tensor index ''' 
    return np.tensordot(gcov, arr, axes=(1, 0))

class Tensor1(object):
    ''' 
    Rank 1 tensor 
    
    Arguments
    arr: Tensor elements; must have shape (4,)
    idx: Index of tensor elements; either COV or CON
    
    Attr:
    cov: Rank (1, 0) tensor
    con: Rank (0, 1) tensor
    rank: Tensor rank
    '''
    def __init__(self, arr, idx=COV):
        
        assert arr.shape == (4,), "Array shape %s does not match rank 1 tensor shape (4,)" % str(arr.shape)
        
        if idx == COV:
            self.cov = arr
            self.con = raize(arr)
        elif idx == CON:
            self.con = arr
            self.cov = lower(arr)
        self.sq = np.inner(self.cov, self.con)
            
        self.rank = 1
        
    def __repr__(self):
        
        return "COV: (%.3g, %.3g, %.3g, %.3g)\nCON: (%.3g, %.3g, %.3g, %.3g)" % (self.cov[0], self.cov[1], self.cov[2], self.cov[3], self.con[0], self.con[1], self.con[2], self.con[3])
        
class Tensor2(object):
    ''' 
    Rank 2 tensor 
    
    Arguments
    arr: Tensor elements; must have shape (4, 4)
    idx: Index of tensor elements; either COV, MIX, or CON
    
    Attr:
    cov: Rank (2, 0) tensor
    mix: Rank (1, 1) tensor
    con: Rank (0, 2) tensor
    rank: Tensor rank
    '''
    def __init__(self, arr, idx=COV):
        
        assert arr.shape == (4, 4), "Array shape %s does not match rank 2 tensor shape (4, 4)" % str(arr.shape)
        
        if idx == COV:
            self.cov = arr
            self.mix = raize(arr)
            self.con = raize(arr)
        elif idx == MIX:
            self.mix = arr
            self.cov = lower(arr)
            self.con = raize(arr)
        elif idx == CON:
            self.con = arr
            self.mix = lower(arr)
            self.cov = lower(arr)
            
        self.rank = 2
        
    def __repr__(self):
        
        return "    COV\n(%.3g, %.3g, %.3g, %.3g)\n(%.3g, %.3g, %.3g, %.3g)\n(%.3g, %.3g, %.3g, %.3g)\n(%.3g, %.3g, %.3g, %.3g)" % (self.cov[0, 0], self.cov[0, 1], self.cov[0, 2], self.cov[0, 3], self.cov[1, 0], self.cov[1, 1], self.cov[1, 2], self.cov[1, 3], self.cov[2, 0], self.cov[2, 1], self.cov[2, 2], self.cov[2, 3], self.cov[3, 0], self.cov[3, 1], self.cov[3, 2], self.cov[3, 3])
        


In [637]:
# constants
Gamma = 5/3 # adiabatic index
alpha = np.sqrt(-1 / gcov[0, 0]) # lapse
beta = alpha**2 * gcon[0, 1:] # shift

n = Tensor1(np.array([-alpha, 0, 0, 0]), idx=COV) # normal observer 4-velocity
j = Tensor2(gcov + np.outer(n.cov, n.cov), idx=COV) # projection tensor onto space normal to normal observer

def proj(tensor):
    ''' Project a tensor onto the space normal to the normal observer '''
    return Tensor1(np.tensordot(j.mix, tensor.con, axes=(1, 0)), idx=CON)
    

In [638]:
def forward(rho0, ug, utilde, B):
    
    p = (Gamma - 1) * ug # pressure
    w = rho0 + ug + p # enthalpy
    gamma = np.sqrt(1 + np.sum(gcov[1:, 1:] * np.outer(utilde, utilde))) # lorentz factor
    
    u = Tensor1(np.concatenate(((gamma / alpha,), utilde - alpha * gamma * gcon[0, 1:])), idx=CON) # 4-velocity
    h = Tensor2(gcov + np.outer(u.cov, u.cov), idx=COV) # projection tensor onto the space normal to the 4-velocity
    b = Tensor1(np.tensordot(h.mix, B.con / gamma, axes=(1, 0)), idx=CON)
    
    D = gamma * rho0
    Q = Tensor1(gamma * (w + b.sq) * u.cov - (p + b.sq/2) * n.cov + np.inner(n.cov, b.con) * b.cov, idx=COV)
    
    return D, Q, B
    

In [668]:
def backward_1dw(D, Q, B):
    ''' 1dw inversion '''
    Qtilde = proj(Q)

    def calc_vsq(W):

        vsq = (Qtilde.sq * W**2 + np.inner(Q.cov, B.con)**2 * (B.sq + 2*W)) / ((B.sq + W)**2 * W**2)
        return vsq

    def calc_p(W):

        vsq = calc_vsq(W)
        gamma = np.sqrt(1 / (1 - vsq))
        rho0 = D / gamma
        w = W / gamma**2
        ug = (w - rho0) / Gamma
        p = (Gamma - 1) * ug
        return p

    def func(W):

        res = -np.inner(Q.cov, n.con) - B.sq * (1 + calc_vsq(W)) / 2 + np.inner(Q.cov, B.con)**2 / (2*W**2) - W + calc_p(W, )
        return res

    W = newton(func, x0=1)
    
    vsq = calc_vsq(W)
    gamma = np.sqrt(1 / (1 - vsq))
    
    rho0 = D / gamma
    w = W / gamma**2
    ug = (w - rho0) / Gamma
    p = (Gamma - 1) * ug
    utilde = gamma / (W + B.sq) * (Qtilde.con[1:] + np.inner(Q.cov, B.con) * B.con[1:] / W)
    
    return rho0, ug, utilde, B


In [682]:
rho0 = 0.1
ug = 0.2
utilde = np.array([1.1, 0, 0])
B = Tensor1(np.array([0, 0, 0, 0]), idx=CON)

D, Q, _ = forward(rho0, ug, utilde, B)
rho0_new, ug_new, utilde_new, _ = backward_1dw(D, Q, B)

print("rho0: %.3g --> %.3g" % (rho0, ug))
print("ug: %.3g --> %.3g" % (ug, ug_new))

Newton's method took 3 iterations to converge.
rho0: 0.1 --> 0.2
ug: 0.2 --> 0.2
